<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/main/workflows/TotalSegmentator/Notebooks/uploadPerFrameToBigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **This Notebook decompresses the csv.lz4 files containing general, shape and first order radiomics features and pushes the data to biqquery table**

### **Installing Packages**

In [ ]:
%%capture
!apt-get install lz4

### **Importing Packages**

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import ast
import pandas as pd
auth.authenticate_user()

In [ ]:
project_id='my project'
client = bigquery.Client(project=project_id)

### **Read the Terra data table with links to output files**

In [ ]:
radiomics_terra_data_table=pd.read_table('path to tsv file with output links')
radiomics_terra_data_table

### **Decompress the csv.lz4, Read csv and load data to bigquery**

In [ ]:
parquet_files= radiomics_terra_data_table.parquet_radiomicsJsonToDataFrame.to_list()
parquet_files

### **Choose a bigquery table**

In [ ]:
bq_table_name="project.dataset.table"

In [ ]:
for parquet_file in parquet_files:
  !rm raw_radiomics.parquet
  !gsutil cp $parquet_file .
  json_radiomics_df= pd.read_parquet(parquet_file)
  json_radiomics_df.columns = json_radiomics_df.columns.str.replace('.', '_')
  # Now load the dataframe into BigQuery
  client.load_table_from_dataframe(json_radiomics_df, bq_table_name, job_config=bigquery.LoadJobConfig(autodetect=True))

